# Computer Vision

### In this tutorial, we are going to learn how to take a photo with a webcam attached to our Pynq Board, display this image in our Jupyter Notebook, and then detect faces in the photo. For extra credit, we will use a neural net to predict the objects in a photo.

## Part 1: Setup

### 1.1 Load Overlay
Our PYNQ Board has a Field Programmable Gate Array (FPGA) on it that must be programmed before we begin using the board. We apply designs called overlays that we can design however we want. For the first part of this tutorial, we are going to use the pre-built base overlay.

In [ ]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
base = BaseOverlay("base.bit")

### 1.2 Import Libraries
Here we tell the board what libraries we want to use in our project.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
from IPython import display

### 1.3 Initialize Webcam
After plugging in our USB webcam, we must tell the board what size images it is going to be recording and sending to the board. We also need to create a python object that will store the images we read from the webcam.

In [ ]:
#create object
videoIn = cv2.VideoCapture(0)

#set input width and height
input_frame_width = 640
input_frame_height = 480
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, input_frame_width);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, input_frame_height);

#check to ensure the webcam is open
if(videoIn.isOpened()):
    print('camera is ready')
else:
    print('error starting camera, run this cell again')

## Part 2: Face Detection

### 2.1 Take Photo
Once we have our board setup and webcam working, it is time to use it! using videoIn.read(), we can read what the webcam is looking at at any given moment. This function has two return values. The first tells us if we successfully read an image or not. True means we did, false means there was a problem. The second return value is a frame object, which is the actual image we read.

In [ ]:
#read frame
success, frame = videoIn.read()
#if there was an error, tell us!
if (success != True):
    print("Video Read Error")

### 2.2 Display Photo
Using matplotlib imported earlier, we can display our image right in our Jupyter notebook

In [ ]:
#display input
plt.imshow(frame[:,:,[2,1,0]])
plt.show()

### 2.3 Look for Faces
Now that we can take a photo, let's have our board look for faces in our photo. We are going to use something called a Haar Cascade Classifier. This is a model that has been training to understand what a face looks like. It has already been built and trained, so we simply need to load it and pass our image to it. Below is what the model is looking for when it tries to find faces, can you trick it?
![HaarClassifier](data/haar.jpg)

In [ ]:
#create our own instance of the classifier
face_cascade = cv2.CascadeClassifier(
    '/home/xilinx/jupyter_notebooks/base/video/data/'
    'haarcascade_frontalface_default.xml')

#our classifier prefers to work on a gray image
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.3, 5)

#draw a rectangle around any faces we find
for (x,y,w,h) in faces:
    cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = frame[y:y+h, x:x+w]

### 2.4 Show our image again with the face detection

In [ ]:
plt.imshow(frame[:,:,[2,1,0]])
plt.show()

### 2.5 Look for Eyes
The board also have another classifier for detecting features that look like eyes. We will do the same thing we did with the face classifier for the eye classifier.

In [ ]:
#create our own instance of the classifier
eye_cascade = cv2.CascadeClassifier(
    '/home/xilinx/jupyter_notebooks/base/video/data/'
    'haarcascade_eye.xml')

eyes = eye_cascade.detectMultiScale(roi_gray)
for (ex,ey,ew,eh) in eyes:
    cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

### 2.6 Show image with eye and face detection

In [ ]:
plt.imshow(frame[:,:,[2,1,0]])
plt.show()

### 2.7 Counting Faces
The faces our program found are contained in an array. This stores the location of the faces within the image. If we want to count the faces found in the image, we can simply print the length of this array.

In [ ]:
print(len(faces))

## Part 3: Guess the object with a neural net

### 3.1 Import a Neural Network

In [ ]:
! sudo pip3 install git+https://github.com/Xilinx/QNN-MO-PYNQ.git

In [ ]:
import qnn
from qnn import Dorefanet
from qnn import utils
import os, pickle, random
from PIL import Image

### 3.2 Take Another Image

In [ ]:
success, frame2 = videoIn.read()

#if there was an error, tell us!
if (success != True):
    print("Video Read Error")
    
#display the image
plt.imshow(frame2[:,:,[2,1,0]])
plt.show()   

#save the image just taken to memory
cv2.imwrite("photo.jpg", frame2)

videoIn.release()

### 3.3 Set Up the Object Classifier
This classifier has been trained to recognize a wide variety of objects. It requires us to reconfigure the FPGA on the board (replacing the base overlay with a custom overlay that will help accelerate the classification). This is automatically done with the init_accelerator() function

In [ ]:
classifier = Dorefanet()
classifier.init_accelerator()
net = classifier.load_network(json_layer="/usr/local/lib/python3.6/dist-packages/qnn/params/dorefanet-layers.json")

conv0_weights = np.load('/usr/local/lib/python3.6/dist-packages/qnn/params/dorefanet-conv0.npy', encoding="latin1").item()
fc_weights = np.load('/usr/local/lib/python3.6/dist-packages/qnn/params/dorefanet-fc-normalized.npy', encoding='latin1').item()

with open("/home/xilinx/jupyter_notebooks/qnn/imagenet-classes.pkl", 'rb') as f:
    classes = pickle.load(f)
    names = dict((k, classes[k][1].split(',')[0]) for k in classes.keys())
    synsets = dict((classes[k][0], classes[k][1].split(',')[0]) for k in classes.keys())
    
conv0_W = conv0_weights['conv0/W']
conv0_T = conv0_weights['conv0/T']

### 3.4 Run the Image through the Classifier
Once our classifier is set up, we input our image to the neural net.

In [ ]:
img, img_class = classifier.load_image("photo.jpg")

# 1st convolutional layer execution, having as input the image and the trained parameters (weights)
conv0 = utils.conv_layer(img, conv0_W, stride=4)
# The result in then quantized to 2 bits representation for the subsequent HW offload
conv0 = utils.threshold(conv0, conv0_T)


# Compute offloaded convolutional layers
out_dim = net['merge4']['output_dim']
out_ch = net['merge4']['output_channels']

conv_output = classifier.get_accel_buffer(out_ch, out_dim);
conv_input = classifier.prepare_buffer(conv0)

classifier.inference(conv_input, conv_output)

conv_output = classifier.postprocess_buffer(conv_output)


# Normalize results
fc_input = conv_output / np.max(conv_output)

# FC Layer 0
fc0_W = fc_weights['fc0/Wn']
fc0_b = fc_weights['fc0/bn']

fc0_out = utils.fully_connected(fc_input, fc0_W, fc0_b)
fc0_out = utils.qrelu(fc0_out)
fc0_out = utils.quantize(fc0_out, 2)

# FC Layer 1
fc1_W = fc_weights['fc1/Wn']
fc1_b = fc_weights['fc1/bn']

fc1_out = utils.fully_connected(fc0_out, fc1_W, fc1_b)
fc1_out = utils.qrelu(fc1_out)

# FC Layer 2
fct_W = fc_weights['fct/W']
fct_b = np.zeros((fct_W.shape[1], ))

fct_out = utils.fully_connected(fc1_out, fct_W, fct_b)

### 3.5 Use Softmax to Estimate the Probability of the Top 5 Most Likely Classes

In [ ]:
# Softmax
out = utils.softmax(fct_out)

# Top-5 results
topn =  utils.get_topn_indexes(out, 5)  
for k in topn: print("class:{0:>20}\tprobability:{1:>8.2%}".format(names[k].lower(), out[k]))

#display results
x_pos = np.arange(len(topn))
plt.barh(x_pos, out[topn], height=0.4, color='g', zorder=3)
plt.yticks(x_pos, [names[k] for k in topn])
plt.gca().invert_yaxis()
plt.xlim([0,1])
plt.grid(zorder=0)
plt.show()

## Part 4: Gesture Sensor

Now we will explore the Grove Gesture Sensor and use it to take pictures with our webcam.

[Grove gesture sensor](http://wiki.seeedstudio.com/Grove-Gesture_v1.0/) on the board.
The gesture sensor can detect 10 gestures as follows:

| Value the Sensor Returns | Gesture            |
|--------------------------|--------------------|
| 0                        | No detection       |
| 1                        | forward            |
| 2                        | backward           |
| 3                        | right              |
| 4                        | left               |
| 5                        | up                 |
| 6                        | down               |
| 7                        | clockwise          |
| 8                        | counter-clockwise  |
| 9                        | wave               |


For this notebook, a PYNQ Arduino shield is also required.
The grove gesture sensor is attached to the I2C interface on the shield. 
This grove sensor should also work with PMOD interfaces on the board.

### 4.1 Import Libraries

In [ ]:
from pynq.lib.arduino import Grove_Gesture
from pynq.lib.arduino import ARDUINO_GROVE_I2C

sensor = Grove_Gesture(base.ARDUINO, ARDUINO_GROVE_I2C)

### 4.2 Set speed
There are currently 2 modes available for users to use: `far` and `near`.
The corresponding fps are 120 and 240, respectively.
For more information, please refer to [Grove gesture sensor](http://wiki.seeedstudio.com/Grove-Gesture_v1.0/).

In [ ]:
sensor.set_speed(240)

### 4.3 Read gestures
The following code will read 10 gestures within 30 seconds. 
Try to change your gesture in front of the sensor and check the results.

In [ ]:
from time import sleep

for i in range(10):
    print(sensor.read_gesture())
    sleep(3)